In [ ]:
# this is how you would download the files directly using a notebook. it is a large dataset and will take awhile. i used google
# drive and processed the files in a colab notebook to share with the team.

!wget https://fileshare.eoir.justice.gov/FOIA-TRAC-Report.zip

In [ ]:
import pandas as pd

In [ ]:
# these three .csv files contain the bulk of the information you'll need to get started. import from your chosen path. 

cases_table = '../A_TblCase.csv'
proceedings_table = '../B_TblProceeding.csv'
judges_table = '../tblLookupJudge.csv'

In [ ]:
# this is how you might process the judges table to create a dictionary
# of judge names and codes along with a reverse dictionary.


judges_cols = ['JUDGE_CODE', 'JUDGE_NAME', 'blnActive']

judges_df = pd.read_csv(judges_table, sep=r'\t', error_bad_lines=False, usecols=judges_cols)

active_judges_df = judges_df[judges_df['blnActive'] == 1]

# creating a dict of judges names and codes as a reference to check 

judges_dict = {}

for code, judge in zip(judges_df['JUDGE_CODE'], judges_df['JUDGE_NAME']):
  if ',' in judge:
    clean_name = judge.strip().upper()
    judges_dict[code] = clean_name   

# creating a reverse dictionary for judge name : code
reverse_judge_dict = {v:k for k, v in judges_dict.items()}

In [ ]:
# this is how you might process the cases table. nrows was used for initial inspection
# you can change the chunksize and date of entry filter as you see fit

cases_cols = ['IDNCASE', 'ALIEN_STATE', 'NAT', 'DATE_OF_ENTRY']

chunksize = 500
nrows = 100

case_chunks = []

for chunk in pd.read_csv(cases_table, sep=r'\t', error_bad_lines=False, usecols=cases_cols,
                      parse_dates=['DATE_OF_ENTRY'], chunksize=chunksize):
  
  case_chunks.append(chunk[chunk['DATE_OF_ENTRY'] >= '2017'])

# creating a dataframe of the cases table from the list of chunks
cases_df = pd.concat(case_chunks)

In [ ]:
# this is how you might process the proceedings table. again, filter as you see fit.

proceedings_cols = ['IDNCASE', 'CASE_TYPE', 'IJ_CODE', 'HEARING_LOC_CODE', 'HEARING_DATE', 'DEC_TYPE', 'DEC_CODE','ABSENTIA']

proceedings_chunks = []

for chunk in pd.read_csv(proceedings_table, sep=r'\t', usecols=proceedings_cols, error_bad_lines=False, 
                             parse_dates=['HEARING_DATE'], chunksize=chunksize):
  
  proceedings_chunks.append(chunk[chunk['HEARING_DATE'] >= '2017'])

# creating a dataframe of the proceedings table from the list of chunks

proceedings_df = pd.concat(proceedings_chunks)

## i would merge proceedings and cases and go from there. the above removes a few hurdles and initial frustrations! 